# 1. Citirea datelor de intrare

In [46]:
import numpy as np
import random
import math
import PySimpleGUI as sg

In [47]:
a = 0
b = 0
l = 0
n = 0

In [70]:
input_type = input("Introduceti tipul inputului(tastatura/fisier)")
if input_type == "tastatura":
    validation = read_stdin()
elif input_type == "fisier":
    validation = read_file()
    
if validation == 1:
    draw_interface()

Introduceti tipul inputului(tastatura/fisier)fisier
[ 2.25    3.      9.0625  3.     13.     24.    ]
[ 9.     35.0625 61.    ]


In [49]:
def read_stdin():
    global a, b, n
    n = input("Dimensiunea matricei: ")
    n = int(n)
    a_aux = np.array([])
    val = 0
    index = 0
    negative_flag = 1
    decimal_flag = 0
    for i in range(0, n):
        row = input()
        for character in row:
            if character == '-':
                negative_flag = -1
            elif character == '.':
                decimal_flag = 1
            else:
                if index > 0 and index % n == 0:
                    index += 1
                    val = int(character)
                elif character != ' ':
                    if decimal_flag == 0:
                        val = val*10 + int(character)
                    else:
                        val += (10**(-decimal_flag)) * int(character)
                        decimal_flag += 1
                else:
                    a_aux = np.append(a_aux, val*negative_flag)
                    negative_flag = 1
                    decimal_flag = 0
                    index += 1
                    val = 0

    for i in range(0, n):
        for j in range(0, n):
            if a_aux[i*n+j] != a_aux[j*n+i]:
                print('Matricea nu este simetrica!')
                return -1
                
    a = np.array([])
    for i in range(0, n):
        for j in range(0, n):
            if i >= j:
                a = np.append(a, a_aux[i*n+j])
                
    print(a)
                
    val = 0
    b = np.array([])
    negative_flag = 1
    decimal_flag = 0
    row = input("Vectorul b:")
    for character in row:
        if character == '-':
            negative_flag = -1
        elif character == '.':
            decimal_flag = 1
        elif character != ' ':
            if decimal_flag == 0:
                val = val*10 + int(character)
            else:
                val += (10**(-decimal_flag)) * int(character)
                decimal_flag += 1
        else:
            b = np.append(b, val*negative_flag)
            index += 1
            val = 0
            negative_flag = 1
            decimal_flag = 0
    b = np.append(b, val*negative_flag)
    print(b)
    return 1

In [50]:
def read_file():
    global a, b, n
    file = open("input.txt", "r")
    file_content = file.read()
    n = int(file_content[0])
    a_aux = np.array([])
    b = np.array([])
    val = 0
    index = 0
    negative_flag = 1
    decimal_flag = 0
    
    for character in file_content[2:]:
        if character == '-':
            negative_flag = -1
        elif character == '.':
            decimal_flag = 1
        elif index > 0 and index % n == 0:
            index += 1
            val = int(character)
        elif character != ' ' and character != '\n':
            if decimal_flag == 0:
                val = val*10 + int(character)
            else:
                val += (10**(-decimal_flag)) * int(character)
                decimal_flag += 1
        else:
            if len(a_aux) < n*n:
                a_aux = np.append(a_aux, val*negative_flag)
            else:
                b = np.append(b, val*negative_flag)
            index += 1
            val = 0
            negative_flag = 1
            decimal_flag = 0
    b = np.append(b, val*negative_flag)
    
    for i in range(0, n):
        for j in range(0, n):
            if a_aux[i*n+j] != a_aux[j*n+i]:
                print('Matricea nu este simetrica!')
                return -1
                
    a = np.array([])
    for i in range(0, n):
        for j in range(0, n):
            if i >= j:
                a = np.append(a, a_aux[i*n+j])
                
    print(a)
    print(b)
    return 1

# 2. Operatii pe matricea optimizata

In [51]:
def matrix_at(matrix, i, j):
    if i >= j:
        index = int((i*(i+1)/2) + j)
        return matrix[index]
    else:
        index = int((j*(j+1)/2) + i)
        return matrix[index]

In [52]:
def matrix_set(matrix, i, j, new_val):
    if i >= j:
        index = int((i*(i+1)/2) + j)
        matrix[index] = new_val
    else:
        index = int((j*(j+1)/2) + i)
        matrix[index] = new_val

In [53]:
def l_matrix_at(matrix, i, j):
    if i >= j:
        index = int((i*(i+1)/2) + j)
        return matrix[index]
    else:
        return 0

# 3. Descompunerea Cholesky

In [54]:
def compute_l(a, b, n):
    l = np.array([])
    for i in range(0, int(n*(n+1)/2)):
        l = np.append(l, 0)
    for i in range(0, n):
        compute_l_column_p(i, l, a, b, n)
    return l

In [55]:
def compute_l_column_p(column, l, a, b, n):    
    suma = 0
    for i in range(0, column):
        suma += l_matrix_at(l,column, i) * l_matrix_at(l,column, i)
    val = math.sqrt(matrix_at(a, column, column) - suma)
    matrix_set(l, column, column, val)
    
    
    for i in range(column+1, n):
        suma = 0
        for j in range(0, column):
            suma += l_matrix_at(l, i, j) * l_matrix_at(l, column, j)
            
        val = matrix_at(a, i, column) - suma
        val = val / l_matrix_at(l, column, column)
        matrix_set(l, i, column, val)

In [56]:
def print_l(matrix):
    global n
    for i in range(0,n):
        for j in range(0, n):
            print(l_matrix_at(matrix,i,j), end = ' ')
        print('')

# 4. Det(A) = Det(L) * Det(LT) = Det(L)^2

In [57]:
def get_determinant(matrix):
    determinant = 1
    for i in range(0, n):
        determinant *= l_matrix_at(matrix,i, i)
        determinant *= l_matrix_at(matrix,i, i)
    return determinant

# 5. Rezolvarea sistemului Ax = B

# 5.1. Ly = B, unde L este inferior triunghiulara

In [58]:
def get_y(l, n, b):
    y = np.array([])
    for i in range(0, n):
        y = np.append(y, 0)
    for i in range(0, n):
        suma = 0
        for j in range(0, i):
            suma += (l_matrix_at(l,i,j) * y[j])
        y[i] = (b[i] - suma) / l_matrix_at(l,i,i)
    return y

# 5.2. LT x = y, unde LT este superior triunghiulara 

In [59]:
def get_x(l, n, y):
    x = np.array([])
    for i in range(0, n):
        x = np.append(x, 0)
    for i in range(n-1, -1, -1):
        suma = 0
        for j in range(i+1, n):
            suma += (l_matrix_at(l,j,i) * x[j])
        x[i] = (y[i] - suma) / l_matrix_at(l,i,i)
    return x

# 6. Acuratetea solutiei

In [60]:
def print_error(a, n, x, b):
    error = 0
    for i in range(0, n):
        suma = 0
        for j in range(0, n):
            suma += matrix_at(a, i, j) * x[j]
        error += (abs(suma - b[i]) * abs(suma - b[i]))
    return error

# 7. Solutiile folosind biblioteci deja existente

# 7.1. Descompunerea Cholesky (calculul lui L)

In [61]:
def get_l_numpy(a, n, b):
    complete_matrix = np.empty((0,n))
    for i in range(0, n):
        row = np.array([[]])
        for j in range(0, n):
            row = np.append(row, matrix_at(a,i,j))
        complete_matrix = np.append(complete_matrix, [row], axis=0)
    l_numpy = np.linalg.cholesky(complete_matrix)
    return l_numpy


In [62]:
def get_l_string(a, n, b):
    final_string = ''
    for i in range(0, n):
        final_string += (np.array2string(get_l_numpy(a, n, b)[i]) + ' ')
    return final_string

# 7.2. Aflarea lui x din ecuatia liniara matriciala Ax = b

In [63]:
def solve_linear_matrix_equation_numpy(a, n, b):
    complete_matrix = np.empty((0,n))
    for i in range(0, n):
        row = np.array([[]])
        for j in range(0, n):
            row = np.append(row, matrix_at(a,i,j))
        complete_matrix = np.append(complete_matrix, [row], axis=0)
    x = np.linalg.solve(complete_matrix, b)
    return x


# 8. Aproximarea inversei matricei A

# 8.1. Fara biblioteca

In [64]:
def get_inverse_matrix(a, n):
    complete_matrix = np.empty((0,n))
    for i in range(0, n):
        b = np.array([])
        for j in range(0, n):
            if j == i:
                b = np.append(b, 1)
            else:
                b = np.append(b, 0)
        y = get_y(l, n, b)
        x = get_x(l, n, y)
        complete_matrix = np.append(complete_matrix, [x], axis=0)

    partial_matrix = np.array([])
    for i in range(0, n):
        for j in range(0, n):
            if i >= j:
                partial_matrix = np.append(partial_matrix, complete_matrix[i][j])
    return partial_matrix
    

# 8.2. Cu Numpy

In [65]:
def get_inverse_matrix_numpy(a, n):
    complete_matrix = np.empty((0,n))
    for i in range(0, n):
        row = np.array([[]])
        for j in range(0, n):
            row = np.append(row, matrix_at(a,i,j))
        complete_matrix = np.append(complete_matrix, [row], axis=0)
    return np.linalg.inv(complete_matrix)


In [66]:
def get_inverse_as_string(a, n):
    final_string = ''
    for i in range(0, n):
        final_string += (np.array2string(get_inverse_matrix_numpy(a,n)[i]) + ' ')
    return final_string

# 8.3 Comparatie

In [67]:
inv = get_inverse_matrix(a, n)
inv2 = get_inverse_matrix_numpy(a, n)

def compare_matrices(a, b):
    complete_matrix = np.empty((0,n))
    for i in range(0, n):
        row = np.array([[]])
        for j in range(0, n):
            row = np.append(row, matrix_at(a,i,j))
        complete_matrix = np.append(complete_matrix, [row], axis=0)
        
    return np.linalg.norm(complete_matrix - b)


# Interfata grafica

In [68]:
def draw_interface():
    global l
    layout = [[sg.Button("OBTINE L DIN DESCOMPUNEREA CHOLESKY")],
              [sg.Text(size=(100, 1), key="L")],
              
              [sg.Button("OBTINE L DIN DESCOMPUNEREA CHOLESKY CU BIBLIOTECA NUMPY")],
              [sg.Text(size=(100, 1), key="LNUMPY")],

              [sg.Button("OBTINE DETERMINANTUL")],
              [sg.Text(size=(100, 1), key="DET")],

              [sg.Button("CALCULEAZA x DIN Ax = B")],
              [sg.Text(size=(100, 1), key="X")],
              
              [sg.Button("CALCULEAZA x DIN Ax = B CU BIBLIOTECA NUMPY")],
              [sg.Text(size=(100, 1), key="XNUMPY")],
              
              [sg.Button("OBTINE ACURATETEA SOLUTIEI FATA DE CEA CU NUMPY")],
              [sg.Text(size=(100, 1), key="ACC")],

              [sg.Button("CALCULEAZA INVERSA")],
              [sg.Text(size=(100, 1), key="INV")],
              
              [sg.Button("CALCULEAZA INVERSA CU NUMPY")],
              [sg.Text(size=(100, 1), key="INVNUMPY")],
              
              [sg.Button("OBTINE ACURATETEA INVERSELOR")],
              [sg.Text(size=(100, 1), key="ACCINV")],

              [sg.Button("EXIT")]
             ]
    window = sg.Window("Calcul numeric - Laborator 2", layout)

    while True:
        event, values = window.read()

        if event == "EXIT":
            break
        elif event == "OBTINE L DIN DESCOMPUNEREA CHOLESKY":
            window["L"].update(compute_l(a, b, n))
            l = compute_l(a, b, n)
        elif event == "OBTINE L DIN DESCOMPUNEREA CHOLESKY CU BIBLIOTECA NUMPY":
            window["LNUMPY"].update(get_l_string(a, n, b))
        elif event == "OBTINE DETERMINANTUL":
            window["DET"].update(get_determinant(l))
        elif event == "CALCULEAZA x DIN Ax = B":
            window["X"].update(get_x(l, n, get_y(l, n, b)))
        elif event == "CALCULEAZA x DIN Ax = B CU BIBLIOTECA NUMPY":
            window["XNUMPY"].update(np.array2string(solve_linear_matrix_equation_numpy(a, n, b)))
        elif event == "CALCULEAZA INVERSA":
            window["INV"].update(get_inverse_matrix(a, n))
        elif event == "CALCULEAZA INVERSA CU NUMPY":
            window["INVNUMPY"].update(get_inverse_as_string(a, n))
        elif event == "OBTINE ACURATETEA SOLUTIEI FATA DE CEA CU NUMPY":
            window["ACC"].update(print_error(a, n, get_x(l, n, get_y(l, n, b)), b))
        elif event == "OBTINE ACURATETEA INVERSELOR":
            window["ACCINV"].update(compare_matrices(get_inverse_matrix(a, n), get_inverse_matrix_numpy(a, n)))
    
    window.close()